In [1]:
!pip install dash

In [2]:
import json
import pandas as pd
import numpy as np

In [3]:
import plotly.express as px
import plotly.graph_objects as go

import dash
from dash import html, dcc
from dash.dependencies import Input, Output

c:\Users\wuensche\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


* **Aufgabe 1**
  * Für ein ausgewähltes Produkt Abhängigkeit von 'price_level' darstellen mit strip-plot
  * Für ein ausgewähltes Produkt Abhängigkeit von 'advertising' darstellen mit strip-plot
  * Für ein ausgewähltes Produkt Abhängigkeit von 'advertising' + 'price_level' darstellen mit strip-plot
* **Aufgabe 2**
  *  Für alle Produkte Abhängigkeit von 'advertising' darstellen

### Load the data

In [4]:
# df_upcstoreweek = pd.read_csv('../data/relrevenue_per_storeupcweek.csv')
# df_upcstore = pd.read_csv('../data/relrevenue_per_upcstore.csv')
# df_upc = pd.read_csv('../data/relrevenue_per_upc.csv')
transaction_df = pd.read_csv("../data/dunnhumby_BatF_transactions.csv")

Datumsstring zu Datum konvertieren (https://strftime.org/)

In [5]:
pd.to_datetime(transaction_df.WEEK_END_DATE, format="%d-%b-%y").head(2)

0   2009-01-14
1   2009-01-14
Name: WEEK_END_DATE, dtype: datetime64[ns]

In [6]:
transaction_df.WEEK_END_DATE = pd.to_datetime(transaction_df.WEEK_END_DATE, format="%d-%b-%y")

In [7]:
transaction_df = transaction_df[~transaction_df.PRICE.isnull()]

In [8]:
transaction_df['price_diff'] = transaction_df.BASE_PRICE - transaction_df.PRICE

In [9]:
transaction_df['price_direction'] = np.sign(transaction_df.price_diff)
transaction_df['price_level'] = transaction_df.price_direction.map({0: 'regular', -1: 'increase', 1: 'discounted'})
# transaction_df.head(2)

# **Aufgabe 1**
Betrachten Sie einzelnes Produkt (z.B. `1600027527`). Analysieren Sie visuell die Wochenumsätze nicht nur in Abhängigkeit vom Preis, sondern insbesondere auch von den getätigten Marketingmaßnahmen (`Feature`, `Display`, ...)

In [10]:
def plot_specProd_priceLvl(upc):
    fig = px.strip(transaction_df[transaction_df.UPC == upc], 
         y='PRICE', 
         x='WEEK_END_DATE', 
         color='price_level'
    )
    return fig
plot_specProd_priceLvl(1600027527)

c:\Users\wuensche\Anaconda3\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


### Plot Funktionen (mit test)

In [11]:
# def plot_allupc():
#     fig = px.strip(
#         df_upc.set_index('marketing').loc[['N', 'T', 'D', 'F', 'FD']].reset_index(), 
#         y = 'relative_revenue', 
#         x = 'marketing',
#         #color = 'upc',
#         custom_data = ['upc'],
#         title = 'Effect per UPC'
#     )
#     return fig
# plot_allupc()

In [12]:
# def plot_upcstore(upc):
#     fig = px.strip(
#         df_upcstore[df_upcstore.upc == upc], 
#         y = 'relative_revenue', 
#         x = 'marketing',
#         #color = 'store_id',
#         category_orders = {'marketing': ['N', 'T', 'D', 'F', 'FD']},
#         title = f'Effect for upc {upc} per Store',
#         custom_data = ['upc', 'store_id']
#     )
#     return fig
    
# plot_upcstore(3800039118)

In [13]:
# def plot_upcstoreweek(upc, store_id):
#     fig = px.strip(
#         df_upcstoreweek[(df_upcstoreweek.UPC == upc) & (df_upcstoreweek.store_id == store_id)], 
#         y = 'relative_revenue', 
#         x = 'date', 
#         color = 'marketing',
#         title = f'Weekly UPC: {upc} @ {store_id}'
#     )
#     return fig
# plot_upcstoreweek(3800039118, 367)

In [14]:
# def create_default_figure():
#     # You can customize this as per your need. Here's an example:
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=[], y=[])) # Empty Scatter Plot
#     fig.update_layout(
#         title="No data selected",
#         xaxis_title="X-axis",
#         yaxis_title="Y-axis"
#     )
#     return fig

# create_default_figure()

## Create Dashboard

In [15]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Marketing Effects (relative Revenue)"),

    # Komponente zum Anzeigen der Callbackdaten
    html.Div([
        dcc.Markdown("""**Click Data**"""),
        html.Pre(id='jsonVis'), # Pre-formatted text element
    ]),
    
    # Hauptgrafik
    dcc.Graph(
        id='main_graph',
        figure=plot_specProd_priceLvl(1600027527)
    ),
    
    # # Zoomstufe 1
    # dcc.Graph(
    #     id='storeupc_graph'
    # ),

    # # Komponente zum Anzeigen der Callbackdaten
    # html.Div([
    #     dcc.Markdown("""**Click Data**"""),
    #     html.Pre(id='jsonVis2'), # Pre-formatted text element
    # ]),
    
    # # Zoomstufe 2
    # dcc.Graph(
    #     id='storeupcweek_graph'
    # ),
])

# ===================================
# Callbacks
# ===================================

@app.callback(
    Output(component_id='jsonVis', component_property='children'), # updates the text in the jsonVis component 
    Input(component_id='main_graph', component_property='clickData') # whenever a point on the main_graph is clicked.
)
def display_click_data(clickData): #  It displays the click data in JSON format.
    return json.dumps(clickData, indent=2)

# @app.callback(
#     Output(component_id='jsonVis2', component_property='children'), # updates the text in the jsonVis component 
#     Input(component_id='storeupc_graph', component_property='clickData') # whenever a point on the main_graph is clicked.
# )
# def display_click_data(clickData): #  It displays the click data in JSON format.
#     return json.dumps(clickData, indent=2)

# @app.callback(
#     Output(component_id='storeupc_graph', component_property='figure'), # updates the figure of the storeupc_graph
#     Input(component_id='main_graph', component_property='clickData') # whenever there is a click on a main_graph element
# )
# def update_fig2(clickData):
#     if clickData:
#         upc = clickData['points'][0]['customdata'][0]
#         return plot_upcstore(upc)
#     return create_default_figure()

# @app.callback(
#     Output(component_id='storeupcweek_graph', component_property='figure'),
#     Input(component_id='storeupc_graph', component_property='clickData'))
# def update_fig3(clickData):
#     if clickData:
#         upc = clickData['points'][0]['customdata'][0]
#         store_id = clickData['points'][0]['customdata'][1]
#         return plot_upcstoreweek(upc, store_id)
#     return create_default_figure()

c:\Users\wuensche\Anaconda3\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Open Dashboard at `plot.{container_id}.laboration.hochschule-stralsund.de`
You find the container_id in the address of you browser

In [16]:
app.run(jupyter_mode="tab", debug=True, host='localhost')

Dash app running on http://localhost:8050/


<IPython.core.display.Javascript object>